In [85]:
import numpy as np
import pandas as pd
from PIL import Image

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import compute_class_weight

import tensorflow as tf

### https://www.kaggle.com/datasets/dgomonov/new-york-city-airbnb-open-data

In [70]:
data = pd.read_csv('/kaggle/input/new-york-city-airbnb-open-data/AB_NYC_2019.csv')

In [82]:
data

,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,Review Year,Review Month,NEIGH_H__Allerton,...,NEIGH_H__Woodside,NEIGH_G__Bronx,NEIGH_G__Brooklyn,NEIGH_G__Manhattan,NEIGH_G__Queens,NEIGH_G__Staten Island,TYPE__Entire home/apt,TYPE__Private room,TYPE__Shared room,availability
0,40.64749,-73.97237,149,1,9,0.21,6,2018.0,10.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1
1,40.75362,-73.98377,225,1,45,0.38,2,2019.0,5.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1
2,40.68514,-73.95976,89,1,270,4.64,1,2019.0,7.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1
3,40.79851,-73.94399,80,10,9,0.10,1,2018.0,11.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0
4,40.74767,-73.97500,200,3,74,0.59,1,2019.0,6.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38838,40.78099,-73.95366,129,1,1,1.00,1,2019.0,7.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1
38839,40.75104,-73.81459,45,1,1,1.00,6,2019.0,7.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1
38840,40.54179,-74.14275,235,1,1,1.00,1,2019.0,7.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1
38841,40.80787,-73.92400,100,1,2,2.00,1,2019.0,7.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0


In [72]:
data.isnull().sum()

id                                    0
name                                 16
host_id                               0
host_name                            21
neighbourhood_group                   0
neighbourhood                         0
latitude                              0
longitude                             0
room_type                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
last_review                       10052
reviews_per_month                 10052
calculated_host_listings_count        0
availability_365                      0
dtype: int64

In [73]:
data = data.drop(['id','name','host_id','host_name'],axis=1)

In [74]:
data = data.dropna(axis=0).reset_index(drop=True)

In [75]:
{col : data[col].unique() for col in data.columns if data[col].dtypes == 'object'}

{'neighbourhood_group': array(['Brooklyn', 'Manhattan', 'Queens', 'Staten Island', 'Bronx'],
       dtype=object),
 'neighbourhood': array(['Kensington', 'Midtown', 'Clinton Hill', 'East Harlem',
        'Murray Hill', 'Bedford-Stuyvesant', "Hell's Kitchen",
        'Upper West Side', 'Chinatown', 'South Slope', 'West Village',
        'Williamsburg', 'Fort Greene', 'Chelsea', 'Crown Heights',
        'Park Slope', 'Windsor Terrace', 'Inwood', 'East Village',
        'Harlem', 'Greenpoint', 'Bushwick', 'Lower East Side',
        'Prospect-Lefferts Gardens', 'Long Island City', 'Kips Bay',
        'SoHo', 'Upper East Side', 'Prospect Heights',
        'Washington Heights', 'Woodside', 'Flatbush', 'Brooklyn Heights',
        'Carroll Gardens', 'Gowanus', 'Flatlands', 'Cobble Hill',
        'Flushing', 'Boerum Hill', 'Sunnyside', 'DUMBO', 'St. George',
        'Highbridge', 'Financial District', 'Ridgewood',
        'Morningside Heights', 'Jamaica', 'Middle Village', 'NoHo',
        'Ditm

In [76]:
nominals = ['neighbourhood','neighbourhood_group','room_type']
prefs = ['NEIGH_H_','NEIGH_G_','TYPE_']

In [77]:
data['Review Year'] = data['last_review'].apply(lambda x: np.float(x[0:4]))
data['Review Month'] = data['last_review'].apply(lambda x: np.float(x[5:7]))
data = data.drop('last_review',axis=1)

/tmp/ipykernel_32/6426199.py:1: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  data['Review Year'] = data['last_review'].apply(lambda x: np.float(x[0:4]))
/tmp/ipykernel_32/6426199.py:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  data['Review Month'] = data['last_review'].apply(lambda x: np.float(x[5:7]))


In [78]:
for col , prefix in zip(nominals , prefs):
    dummie = pd.get_dummies(data[col],prefix=prefix).astype(np.float)
    data = pd.concat([data,dummie],axis=1)
    data = data.drop(col,axis=1)

/tmp/ipykernel_32/2705950972.py:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dummie = pd.get_dummies(data[col],prefix=prefix).astype(np.float)
/tmp/ipykernel_32/2705950972.py:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dummie = pd.get_dummies(data[col],prefix=prefix).astype(np.float)
/tmp/ipykernel_32/2705950972.py:2: Deprecation

In [79]:
data['availability'] = pd.qcut(data['availability_365'],q=2,labels=[0,1])

In [80]:
data = data.drop('availability_365',axis=1)

In [81]:
y = data['availability']
x = data.drop('availability',axis=1)

In [88]:
#Image.open('/kaggle/input/new-york-city-airbnb-open-data/New_York_City_.png')

In [89]:
scaler = StandardScaler()
x = scaler.fit_transform(x)

In [90]:
x_train , x_test , y_train , y_test = train_test_split(x,y,train_size=0.7,random_state=13)

In [96]:
np.array(y).mean()
x.shape

0.49936925572175167

In [102]:

weights =dict(enumerate( compute_class_weight(
    'balanced',
    classes = y.unique(),
    y=y

        )
    )
)

In [ ]:
inputs = tf.keras.Input(shape=(235,))
x = tf.keras.layers.Dense(64,activation='relu')(inputs)
x = tf.keras.layers.Dense(64,activation='relu')(x)
outputs = tf.keras.layers.Dense(1,activation='sigmoid')(x)

model = tf.keras.Model(inputs=inputs,outputs=outputs)

model.compile(
    optimizer='adam'
    ,loss = 'binary_crossentropy'
    ,metrics=[
        tf.keras.metrics.binary_accuracy
        ,tf.keras.metrics.AUC(name='auc')
        
    ]
)

hist = model.fit(
    x_train,
    y_train,
    validation_split = 0.2,
    class_weight=weights,
    batch_size = 40,
    epochs = 100,
    callbacks= tf.keras.callbacks.ReduceLROnPlateau(),
    verbose=1
)

Epoch 1/100
544/544 [==============================] - 3s 3ms/step - loss: 0.5759 - binary_accuracy: 0.7050 - auc: 0.7737 - val_loss: 0.5422 - val_binary_accuracy: 0.7403 - val_auc: 0.8088 - lr: 0.0010
Epoch 2/100
544/544 [==============================] - 2s 3ms/step - loss: 0.5243 - binary_accuracy: 0.7412 - auc: 0.8151 - val_loss: 0.5319 - val_binary_accuracy: 0.7475 - val_auc: 0.8180 - lr: 0.0010
Epoch 3/100
544/544 [==============================] - 2s 3ms/step - loss: 0.5122 - binary_accuracy: 0.7498 - auc: 0.8246 - val_loss: 0.5251 - val_binary_accuracy: 0.7479 - val_auc: 0.8221 - lr: 0.0010
Epoch 4/100
544/544 [==============================] - 2s 3ms/step - loss: 0.5041 - binary_accuracy: 0.7538 - auc: 0.8299 - val_loss: 0.5158 - val_binary_accuracy: 0.7506 - val_auc: 0.8248 - lr: 0.0010
Epoch 5/100
544/544 [==============================] - 2s 3ms/step - loss: 0.4982 - binary_accuracy: 0.7535 - auc: 0.8344 - val_loss: 0.5118 - val_binary_accuracy: 0.7536 - val_auc: 0.8269 - l